## Simulation of Franck-Condon intensity pattern in emission spectra of I$_2$
Assumes Morse potentials for the excited $B$ and ground $X$ states






In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import ipywidgets as widgets

# for local installation
# sys.path.append("../") # go to parent dir if running notebook directly in sub-folder

# for Google Colab
!git clone http://github.com/tjbarnum13/Fourier-DVR-1D.git
sys.path.insert(0,'/content/Fourier-DVR-1D')

from fourier_DVR_1D import Domain_Fourier_DVR_1D

Cloning into 'Fourier-DVR-1D'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 123 (delta 12), reused 16 (delta 5), pack-reused 93 (from 1)
Receiving objects: 100% (123/123), 590.93 KiB | 6.42 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
# settings
x_min = 2.0
x_max = 10.0
n_DVR = 300 # number of DVR grid points
n_g = 1001 # number of grid points for integration
n_plot = 100 # number of eigenstates for lower state

# constants
c = 137.036 # speed of light
mass = 1822.888486209 # 1 amu in atomic units
energy = 219474.63068 # cm-1 to au
length = 0.52917724900001 # A to au

In [3]:
domain = Domain_Fourier_DVR_1D(x_min, x_max, n_DVR)

In [20]:
def run(origin,De1,De2,v1,v2,xe1,xe2,m1,m2,initial,xrange,yrange):

  ### Calculate reduced mass in au
  mu = m1*m2/(m1+m2)
  mu = mu*mass

  ### set up PECs

  # convert input energy parameters from cm-1 to au
  De1 = De1/energy
  De2 = De2/energy
  v1 = v1/energy
  v2 = v2/energy

  # convert input length parameters from Angstroms to au
  xe1=xe1/length
  xe2=xe2/length

  # calculate "width" of Morse potential
  a1=v1*np.sqrt(mu/(2*De1))
  a2=v2*np.sqrt(mu/(2*De2))

  # Morse potential
  V1 = lambda x: De1 * (1 - np.exp(-a1 * (x - xe1)))**2 # potential energy curve 1
  V2 = lambda x: De2 * (1 - np.exp(-a2 * (x - xe2)))**2 # potential energy curve 2

  ### solve
  E1, E_four1 = domain.solve(mu, V1)
  E2, E_four2 = domain.solve(mu, V2)

  ### evaluate eigenstates on grid
  x = np.linspace(x_min, x_max, n_g)
  psi_x1 = domain.grid(x, E_four1[:,:n_plot])
  psi_x2 = domain.grid(x, E_four2[:,initial-1:initial])

  # normalize all wavefunctions
  def normalize(psi, x):
    N = np.trapezoid((psi)**2,x)
    return (1/N) * psi

  psi_x2=normalize(psi_x2[-1],x)

  for i in range(n_plot):
    psi_x1[i]=normalize(psi_x1[i],x)

  ### Calculate FCFs
  FCF = np.zeros(n_plot)

  for i in range(n_plot):
    FCF[i] = abs(np.trapezoid(psi_x1[i] * psi_x2, x))**2

  # check that FCFs sum to 1, if much less than 1 there is significant overlap with a vibrational state not included
  if FCF.sum() < 0.95:
    print('Warning: Increase n_plot. Significant Franck-Condon overlap is missing.')

  ### Calculate transition energies
  transition_energy = origin-(E1[:n_plot]-E2[initial])*energy

  ### plot simulated spectrum (nm units)
  fig = plt.figure(figsize=[8,4])
  plt.bar(10**7/transition_energy,FCF)
  plt.xlabel('Wavelength (nm)', fontsize=12)
  plt.ylabel('FC Intensity', fontsize=12)
  plt.xlim(xrange)
  plt.ylim(yrange)
  plt.show()

In [21]:
cu = False # turn off continous update for smoother operation

# set up all the sliders to change variables
initial=widgets.IntText(value=28,min=0,max=70,description='Initial Level:',continuous_update=cu)
origin=widgets.FloatText(value=15769.1,min=0,max=100000,step=0.1,description='Band Origin:',continuous_update=cu)
De1=widgets.FloatText(value=12547.3,min=0,max=10000,step=0.1,description='De(X):',continuous_update=cu)
De2=widgets.FloatText(value=4381.2,min=0,max=10000,step=0.1,description='De(B):',continuous_update=cu)
v1=widgets.FloatText(value=214.53,min=0,max=10000,step=0.01,description='ve(X):',continuous_update=cu)
v2=widgets.FloatText(value=125.67,min=0,max=10000,step=0.01,description='ve(B):',continuous_update=cu)
xe1=widgets.FloatText(value=2.6664,min=0.1,max=10,step=0.001,description='re(X):',continuous_update=cu)
xe2=widgets.FloatText(value=3.014,min=0.1,max=10,step=0.001,description='re(B):',continuous_update=cu)
m1=widgets.FloatText(value=126.90447,min=0.1,max=1000,step=0.1,description='m1:',continuous_update=cu)
m2=widgets.FloatText(value=126.90447,min=0.1,max=1000,step=0.1,description='m2:',continuous_update=cu)
xrange=widgets.IntRangeSlider(value=[530,615],min=200,max=1500,step=1,description='X-Range:',continuous_update=cu)
yrange=widgets.FloatRangeSlider(value=[0,0.03],min=0,max=0.1,step=0.005,description='Y-Range:',continuous_update=cu)

# define the output and arrange sliders and plot
out = widgets.interactive_output(run,{'initial':initial,'origin':origin,'De1':De1,'De2':De2,'v1':v1,'v2':v2,'xe1':xe1,'xe2':xe2,'m1':m1,'m2':m2,'xrange':xrange,'yrange':yrange})
controls = widgets.VBox([initial,origin,De1,De2,v1,v2,xe1,xe2,m1,m2,xrange,yrange])
display(widgets.HBox([controls, out]))